In [2]:
import pandas as pd 
import numpy as np
import json
from collections import namedtuple
from os.path import exists

In [3]:
df = pd.read_csv('steam/steam_data.csv')
df.head(1)

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,https://store.steampowered.com/app/945360/Amon...,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...","Overwhelmingly Positive(224,878)- 95% of the 2...","Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-


Parse URLS to uuid

In [4]:
import re
url_list = []

def findid(row):
    result = re.search(string=row, pattern=r'\b/[0-9]*/')
    if result is not None:
        result2 = re.sub(string=result.group(0), pattern=r'/*', repl='')
        if result2 is not None:
            url_list.append(result2)

df['url'].apply(lambda x: findid(x))
steam_ids = pd.DataFrame(url_list)

In [5]:
steam_ids.head()

,0
0,945360
1,730
2,1097150
3,1158310
4,1085660


In [6]:
# from steam import Steam
# import requests
# import json


# file = open('./env.txt')
# STEAM_KEY = file.read() 
# data = []
# steam = Steam(STEAM_KEY)
# uuid='945360'

# def getgamereviews(uuid): 
#     result = requests.get('https://store.steampowered.com/appreviews/'+uuid+'?json=1cursor=20')
#     return result.text

# data.append(getgamereviews(uuid='945360'))
# with open('test.json', 'w', encoding="utf-8") as f:
#     json.dump(data, f)

In [7]:
# comments = []
# comments_dict = {}
# for row in data: 
#     json_line = json.loads(row)
#     for index in range(20):
#         comments.append(json_line['reviews'][index]['review'])
#     comments_dict[uuid] = comments

In [8]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# comments_df = pd.Series(comments_dict, name='comments')
# comments_df

In [9]:
class Field:
    """
    Representational json from steam
    """
    data = []

    def __init__(self):
        self._load_data()

    def _load_data(self) -> None: 
        with open('final_data_new.json', 'r', encoding="utf-8") as file:
            for line in file:
                json_line = json.loads(line,  object_hook=BlankDict)
                for index in range(len(json_line)):
                    if json_line[index]['publisher'] is not None:

                        rec = Record.record(img_url=json_line[index]['img_url'], date=json_line[index]['date'], 
                                            developer=json_line[index]['developer'], publisher=json_line[index]['publisher'], 
                                            full_desc=json_line[index]['full_desc'], 
                                            requirements=json_line[index]['requirements'], popu_tags=json_line[index]['popu_tags'], 
                                            price=json_line[index]['price'], url_info=json_line[index]['url_info'], name=json_line[index]['name'], 
                                            categories=json_line[index]['categories'])
                        self.data.append(rec)
        file.close()

class BlankDict(dict):
    def __missing__(self, key):
        return ''

class Record: 
    """
    Representation of a record
    """
    field = ['img_url', 'date', 'developer', 'publisher', 'full_desc', 'requirements', 'popu_tags', 
             'price', 'url_info', 'name', 'categories']
    record = namedtuple('Record', field)

field = Field()
field.__init__()
print(field.data[0])

Record(img_url='https://steamcdn-a.akamaihd.net/steam/apps/945360/header.jpg?t=1598556351', date='Nov 16, 2018', developer='Innersloth', publisher='Innersloth', full_desc={'sort': 'game', 'desc': "About This Game Play with 4-10 player online or via local WiFi as you attempt to prepare your spaceship for departure, but beware as one or more random players among the Crew are Impostors bent on killing everyone!Originally created as a party game, we recommend playing with friends at a LAN party or online using voice chat. Enjoy cross-platform play between Android, iOS and PC. Win by completing tasks to prepare the ship or ejecting all Impostors. React quickly to undo the Impostor's sabotages. Check the Admin map and Security cameras to keep tabs on other Crewmates. Report any dead bodies immediately to start discussion of who the suspected Impostor is. Call emergency meetings to discuss suspicious behavior. Vote to eject suspected Impostors. Kill crewmates and frame bystanders. Pretend to 

In [10]:
clean_df = pd.DataFrame(field.data, dtype='object', columns=Record.field)
print(clean_df.shape)
clean_df = clean_df.drop_duplicates(subset=['img_url'])

(149642, 11)


In [11]:
clean_df.shape

(64344, 11)

In [12]:
clean_df.head(1)

,img_url,date,developer,publisher,full_desc,requirements,popu_tags,price,url_info,name,categories
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,"{'sort': 'game', 'desc': 'About This Game Play...",{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,{'url': 'https://store.steampowered.com/app/94...,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross..."


In [13]:
clean_df.tail(3)

,img_url,date,developer,publisher,full_desc,requirements,popu_tags,price,url_info,name,categories
74805,https://steamcdn-a.akamaihd.net/steam/apps/134...,Coming Soon,Pathea Games,,"{'sort': 'game', 'desc': 'About This Game Abou...",{'minimum': {'windows': {'processor': ' Intel ...,"[Sports, Multiplayer, Online, Casual, Team-, B...",free,{'url': 'https://store.steampowered.com/app/13...,Super Buckyball Tournament Preseason,"[Online PvPOnline, Co-opFull controller, suppo..."
74806,https://steamcdn-a.akamaihd.net/steam/apps/134...,TBA,Yumenamakon,,"{'sort': 'game', 'desc': 'About This Game Tire...",{'minimum': {'windows': {'processor': ' Intel ...,"[Adventure, Anime, Singleplayer, Pixel, Graphi...",,,Lisa and the Grimoire,[Single-playerSteam Cloud]
74807,https://steamcdn-a.akamaihd.net/steam/apps/134...,"1 Dec, 2020",Carlos Coronado,,"{'sort': 'game', 'desc': 'About This Game 7th ...",{'minimum': {'windows': {'processor': ' Intel ...,"[Psychological, Horror, Horror, Action-, Adven...",,,HORROR TALES: The Beggar,"[Single-playerSteam Achievements Full, control..."


In [15]:
clean_df.reindex()
print(clean_df.shape)

(64344, 11)
